# 🎯 Interview Trainer Agent with IBM Granite & Watsonx
This notebook builds an AI-powered Interview Prep Assistant using IBM Watsonx and Granite models. It supports resume parsing, role-specific question generation, and integration with IBM Cloud Lite services.

### 🔐 Watsonx Connection

In [ ]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your api key: ")
)
client = APIClient(credentials)

In [ ]:
# Set your space ID from Watsonx cloud project
space_id = "your-space-id-here"
client.set.default_space(space_id)

### 📄 Resume Parser

In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        return '\n'.join([page.extract_text() for page in pdf if page.extract_text()])

# Example usage:
# resume_text = extract_text_from_pdf("resume.pdf")
# print(resume_text)

### 🧠 RAG Setup with FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample questions
corpus = [
    'What are your strengths?',
    'Tell me about yourself.',
    'Describe a team challenge you handled.',
    'Why do you want this job?',
    'Where do you see yourself in 5 years?'
]

embeddings = model.encode(corpus)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

### 🤖 Retrieval & AI Response

In [ ]:
def retrieve_question(user_input):
    query_vector = model.encode([user_input])
    D, I = index.search(query_vector, k=2)
    return [corpus[i] for i in I[0]]

# print(retrieve_question('Describe a workplace challenge'))

In [ ]:
# Placeholder Granite response function
def generate_answer(question):
    return f'Model answer for: {question}'

# print(generate_answer('Tell me about yourself.'))

### 🧬 IBM AI Service Function

In [ ]:
params = {"space_id": space_id}

def gen_ai_service(context, params=params, **custom):
    from langchain_ibm import ChatWatsonx
    from ibm_watsonx_ai import APIClient

    question = custom.get('input', 'Tell me about yourself.')
    llm = ChatWatsonx(model="granite-13b-chat-v2", credentials=credentials)
    return llm.invoke(question)

# Example (for deployment use):
# gen_ai_service(None, input='Describe your leadership style.')